In [1]:
# Data Pre-processing
import glob
import json
import random
from tqdm import tqdm
import random
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize

from pdf_extract_text_without_table import DocPdfplumber

pdf_to_json_text_data_path = 'similarity_data.json'

def get_files(file_path):
    files = glob.glob(file_path)
    return files

def get_sentences(data):
    sentences = sent_tokenize(data)
    sentences = [i.strip().replace('\n','').replace('_','').lower() for i in sentences]
    #sentences = "".join(sentences)
    
    return sentences

print("==========")
print(get_files("../data/pdf_docs/*.pdf"))
all_files_data = {}
for each_file_path in tqdm(get_files("../data/pdf_docs/*.pdf")):
    #print(each_file_path)
    doc_pdfplumber = DocPdfplumber(each_file_path)
    data_pages = doc_pdfplumber.get_pdf_page_info()
    #print(data_pages)
    for each_page in data_pages:
        #print(each_page)
        sents = get_sentences(each_page['page_without_table'])
        each_page['sentences'] = sents
        #print("\n=====\n")
    
    all_files_data[each_file_path] = data_pages

with open(pdf_to_json_text_data_path, 'w') as f:
    json.dump(all_files_data, f)

['../data/pdf_docs\\04010045.pdf', '../data/pdf_docs\\ADAMCO_compliance.pdf', '../data/pdf_docs\\AMIND-Contingent Worker Agreement _ revised (1).pdf', '../data/pdf_docs\\CSB_Compliance Policy.pdf', '../data/pdf_docs\\CW9930-Amendment 01 to SOW 11 - Evalueserve.pdf', '../data/pdf_docs\\Evaluaserv SOW 12 Data Science Pilot Team.pdf', '../data/pdf_docs\\Evaluaserv SOW 13 Sr Delivery Manager - Prathibha Das.pdf', '../data/pdf_docs\\Evaluaserv SOW 14 Design Specialist - Rakhi RS.pdf', '../data/pdf_docs\\Evaluaserv SOW 5 Data Science Solution Architect - Varun Sawhney.pdf', '../data/pdf_docs\\Evaluaserv SOW 6 Senior SAS Developer - Lakshmi Nirmala Ambati.pdf', '../data/pdf_docs\\Evaluserve Amendment1 to Variation1 CW1096806.pdf', '../data/pdf_docs\\Global Policy Booklet, POSH & GDPR.PDF', '../data/pdf_docs\\IND-18264-ECard.pdf', '../data/pdf_docs\\Massey_University_Compliance Policy.pdf', '../data/pdf_docs\\Mediclaim Insurance contact Points.pdf', '../data/pdf_docs\\MFA app SOP.pdf', '../dat

100%|██████████| 22/22 [00:45<00:00,  2.09s/it]


In [30]:
import nlpaug.augmenter.word as naw
#import nlpaug.augmenter.char as nac
#import nlpaug.augmenter.sentence as nas
#import nlpaug.flow as nafc
#from nlpaug.util import Action

#synonym_word_substitute_aug = naw.SynonymAug(aug_src='wordnet')
crop_words_aug = naw.RandomWordAug(action='crop')
swap_aug = naw.RandomWordAug(action="swap")
contextual_word_embs_aug = naw.ContextualWordEmbsAug(model_path='roberta-base', action="substitute")
#back_translation_aug = naw.BackTranslationAug(from_model_name='facebook/wmt19-en-de', to_model_name='facebook/wmt19-de-en')
#back_translation,synonym_word_substitute
variation_strategy = ['as_it_is','crop_continous_words','swap_words','contextual_word_embs']

def get_text_variation(text,strategy='as_it_is'):
    #text = 'The quick brown fox jumped over the lazy dog'
    if len(text.replace('\n','').split()) < 4:
        return text
    
    out = text
    if strategy == 'as_it_is':
        out = text
    #elif strategy == 'back_translation':
    #    out = back_translation_aug.augment(text)
    #    print("Original:")
    #    print(text)
    #    print("Augmented Text:")
    #    print(out)
    elif strategy == 'swap_words':
        out = swap_aug.augment(text)
        out = out[0]
        #print("Original:")
        #print(text)
        #print("Augmented Text:")
        #print(out)
    elif strategy == 'crop_continous_words':
        out = crop_words_aug.augment(text)
        out = out[0]
        #print("Original:")
        #print(text)
        #print("Augmented Text:")
        #print(out)
    #elif strategy == 'synonym_word_substitute':
    #    out = synonym_word_substitute_aug.augment(text)
    #    print("Original:")
    #    print(text)
    #    print("Augmented Text:")
    #    print(out)
    elif strategy == 'contextual_word_embs':
        out = contextual_word_embs_aug.augment(text)
        out = out[0]
        #print("Original:")
        #print(text)
        #print("Augmented Text:")
        #print(out)
    
    return out

In [31]:
get_text_variation('The quick brown fox jumps over the lazy dog .',strategy='swap_words')

'The quick fox brown jumps over the lazy dog.'

In [46]:


with open(pdf_to_json_text_data_path) as data_file:
    data_loaded = json.load(data_file)

similar_docs = ["../data/pdf_docs\Evaluaserv SOW 12 Data Science Pilot Team.pdf",
                "../data/pdf_docs\Evaluaserv SOW 13 Sr Delivery Manager - Prathibha Das.pdf",
                "../data/pdf_docs\Evaluaserv SOW 14 Design Specialist - Rakhi RS.pdf",
                "../data/pdf_docs\Evaluaserv SOW 5 Data Science Solution Architect - Varun Sawhney.pdf",
                "../data/pdf_docs\Evaluaserv SOW 6 Senior SAS Developer - Lakshmi Nirmala Ambati.pdf"
                ]

files = list(data_loaded.keys())
dataset_for_model = []
for key in tqdm(files):
    #key = '../data/pdf_docs\\04010045.pdf'
    data = data_loaded[key]
    print(key)

    # pasitive
    for page_data in data:
        for sentence1 in page_data['sentences']:
            if len(sentence1.replace('\n','').split(' '))<4:
                continue

            strategy = random.choice(variation_strategy)
            if strategy == 'as_it_is':
                score = 1
            elif strategy == 'contextual_word_embs':
                score = 0.9
            else:
                score = 0.8
            
            sentence2 = get_text_variation(sentence1,strategy)
            tmp = {'sentence1':sentence1,
                'sentence2':sentence2,
                'score':score,
                'split':'train'}
            dataset_for_model.append(tmp)
    print("========================")
    # negative
    if key in similar_docs:
        exculded_files = [i for i in files if i not in [key]+similar_docs]
    else:
        exculded_files = [i for i in files if i not in [key]]
    
    # excluded sentences
    excluded_sents = []
    for exc_file in exculded_files:
        for exc_page_data in data_loaded[exc_file]:
            exc_sents_filtered = [i for i in exc_page_data['sentences'] if len(i.replace('\n','').split(' '))>=4]
            excluded_sents.extend(exc_sents_filtered)
    print("-------------------------")
    for page_data in data:
        for sentence1 in page_data['sentences']:
            if len(sentence1.replace('\n','').split(' '))<4:
                continue
            tmp = {'sentence1':sentence1,
                'sentence2':random.choice(excluded_sents),
                'score':0.2,
                'split':'train'}
            dataset_for_model.append(tmp)
    print("*************************")
    
    #break

  0%|          | 0/1 [00:00<?, ?it/s]

../data/pdf_docs\04010045.pdf


  0%|          | 0/1 [01:37<?, ?it/s]

-------------------------
*************************


In [51]:
#dataset_for_model[:5]
dataset_for_model_dict = {'a':list(),'b':list(),'c':list(),'d':list()}

for each_rec in dataset_for_model:
    if each_rec['score'] == 1:
        dataset_for_model_dict['a'].append(each_rec)
    elif each_rec['score'] == 0.9:
        dataset_for_model_dict['b'].append(each_rec)
    elif each_rec['score'] == 0.8:
        dataset_for_model_dict['c'].append(each_rec)
    elif each_rec['score'] == 0.2:
        dataset_for_model_dict['d'].append(each_rec)

In [57]:
[{i:len(dataset_for_model_dict[i])} for i in list(dataset_for_model_dict.keys())]

[{'a': 52}, {'b': 65}, {'c': 114}, {'d': 231}]

In [62]:
import numpy as np

def get_train_val_test_index(max_ind=21):
    #a = np.arange(1, 21)
    a = np.array(list(range(0,max_ind)))
    aa = np.split(a, [int(.8 * len(a)), int(.9 * len(a))])
    out = [list(i) for i in aa]

    return out

In [65]:
datasets_for_model_finetune = []
for k in ['a','b','c','d']:
    tr_v_te = get_train_val_test_index(len(dataset_for_model_dict[k]))
    for ind,g in enumerate(tr_v_te):
        if ind==0:
            #print(g)
            #print(dataset_for_model_dict[k])
            for l in [dataset_for_model_dict[k][p] for p in g]:
                l['split'] = 'train'
                datasets_for_model_finetune.append(l)
        elif ind==1:
            for l in [dataset_for_model_dict[k][p] for p in g]:
                l['split'] = 'val'
                datasets_for_model_finetune.append(l)
        elif ind==2:
            for l in [dataset_for_model_dict[k][p] for p in g]:
                l['split'] = 'test'
                datasets_for_model_finetune.append(l)

In [68]:
train = [i for i in datasets_for_model_finetune if i['split']=='train']
val = [i for i in datasets_for_model_finetune if i['split']=='val']
test = [i for i in datasets_for_model_finetune if i['split']=='test']
{'train':len(train),'val':len(val),'test':len(test)}

{'train': 368, 'val': 45, 'test': 49}

In [71]:
train_for_model_dict = {'a':list(),'b':list(),'c':list(),'d':list()}

for each_rec in test:
    if each_rec['score'] == 1:
        train_for_model_dict['a'].append(each_rec)
    elif each_rec['score'] == 0.9:
        train_for_model_dict['b'].append(each_rec)
    elif each_rec['score'] == 0.8:
        train_for_model_dict['c'].append(each_rec)
    elif each_rec['score'] == 0.2:
        train_for_model_dict['d'].append(each_rec)

[{i:len(train_for_model_dict[i])} for i in list(train_for_model_dict.keys())]

[{'a': 6}, {'b': 7}, {'c': 12}, {'d': 24}]

In [72]:
"""
This example loads the pre-trained SentenceTransformer model 'nli-distilroberta-base-v2' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.

Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

#Check if dataset exsist. If not, download and extract  it
#sts_dataset_path = 'datasets/stsbenchmark.tsv.gz'

#if not os.path.exists(sts_dataset_path):
#    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)




# Read the dataset
model_name = 'nli-distilroberta-base-v2'
train_batch_size = 16
num_epochs = 4
model_save_path = 'output/training_stsbenchmark_continue_training-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



# Load a pre-trained sentence transformer model
model = SentenceTransformer(model_name)

2023-07-07 18:43:12 - Load pretrained SentenceTransformer: nli-distilroberta-base-v2
2023-07-07 18:43:15 - Use pytorch device: cpu


In [73]:
# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")

train_samples = []
dev_samples = []
test_samples = []
#with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
#    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    #print(len(reader))
    #count = 0
if True:
    #for row in reader:
    for row in datasets_for_model_finetune:
        #score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
        score = row['score']
        inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)

        if row['split'] == 'dev':
            dev_samples.append(inp_example)
        elif row['split'] == 'test':
            test_samples.append(inp_example)
        else:
            train_samples.append(inp_example)
        
        #count += 1
        #if count >= 100:
        #    break

2023-07-07 18:43:57 - Read STSbenchmark train dataset


In [74]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

In [75]:
#help(train_dataloader.sampler)

In [76]:

# Development set: Measure correlation between cosine score and gold labels
logging.info("Read STSbenchmark dev dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

2023-07-07 18:44:05 - Read STSbenchmark dev dataset
2023-07-07 18:44:05 - Warmup-steps: 11


In [77]:

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/26 [00:00<?, ?it/s]

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3686400 bytes.

In [ ]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

# 1st try

In [ ]:
"""
This examples trains BERT (or any other transformer model like RoBERTa, DistilBERT etc.) for the STSbenchmark from scratch. It generates sentence embeddings
that can be compared using cosine-similarity to measure the similarity.

Usage:
python training_nli.py

OR
python training_nli.py pretrained_transformer_model_name
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import sys
import os
import gzip
import csv

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout



#Check if dataset exsist. If not, download and extract  it
sts_dataset_path = 'datasets/stsbenchmark.tsv.gz'

if not os.path.exists(sts_dataset_path):
    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)



#You can specify any huggingface/transformers pre-trained model here, for example, bert-base-uncased, roberta-base, xlm-roberta-base
model_name = sys.argv[1] if len(sys.argv) > 1 else 'distilbert-base-uncased'

# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = 'output/training_stsbenchmark_'+model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
word_embedding_model = models.Transformer(model_name)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


In [ ]:

# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")

train_samples = []
dev_samples = []
test_samples = []
with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        score = float(row['score']) / 5.0  # Normalize score to range 0 ... 1
        inp_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=score)

        if row['split'] == 'dev':
            dev_samples.append(inp_example)
        elif row['split'] == 'test':
            test_samples.append(inp_example)
        else:
            train_samples.append(inp_example)

In [ ]:

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


logging.info("Read STSbenchmark dev dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

In [ ]:

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

In [ ]:

##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)